In [36]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.multiprocessing as mp
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from datasets import load_dataset
import torch.optim as optim
from torchvision.transforms import transforms

from torch.cuda.amp import  GradScaler
from torch.amp import autocast
import gc
from dataset_convert import AnchorGenerator, FaceDetectionDataset
# device = torch.device("mps")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [77]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
# model = model.features.to(device)

Using cache found in /Users/vipulagarwal/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
dataset = load_dataset("CUHK-CSE/wider_face")
train_dataset = dataset['train'].with_format("torch")
val_dataset = dataset['validation'].with_format("torch")

In [4]:
extractor = Features(model,['3','6', '13','18'])
topdown = FPNetwork(out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction(lambd=10)
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_dataset,anchors)
vali_data = FaceDetectionDataset(val_dataset,anchors)

In [ ]:
# /Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/models

In [5]:
sd_extractor = torch.load('/Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/models/extractor.pt',map_location=torch.device('cpu'))
sd_classifier = torch.load('/Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/models/classifier.pt',map_location=torch.device('cpu'))
sd_bboxregression = torch.load('/Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/models/bboxregression.pt',map_location=torch.device('cpu'))
sd_topdown = torch.load('/Users/vipulagarwal/Documents/AI & ML/Projects/New_FR/version2/faces2.0/models/topdown.pt',map_location=torch.device('cpu'))

extractor.load_state_dict(sd_extractor)
topdown.load_state_dict(sd_topdown)
classifier.load_state_dict(sd_classifier)
bboxregression.load_state_dict(sd_bboxregression)


/var/folders/pc/3dvbqf1x7ts37h_0ccg5yh5r0000gn/T/ipykernel_12899/1582781517.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd_extractor = torch.load('/Users/vipulagarwa

<All keys matched successfully>

In [6]:
def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
    return output

In [7]:
class Forward:
    def __init__(self, extractor, topdown, bboxregression, classifier):
        self.extractor=extractor
        # self.extractor = Features(extractor,['3','6', '13','18'])
        self.topdown = topdown
        self.bboxregression = bboxregression
        self.classifier = classifier
        
    def __call__(self, inp):
        self.raw_features = self.extractor.extract(inp)
        self.newfeatures = self.topdown(self.raw_features)
        output = {}
        for key in list(self.newfeatures.keys()):
            temp = {}
            temp["bbox"] = self.bboxregression(self.newfeatures[key])
            temp["cls"] = self.classifier(self.newfeatures[key])
            output[key] = temp
            self.output = output
        return self.output        

In [8]:
fd= Forward(extractor=extractor,
            topdown=topdown,
            bboxregression=bboxregression,
            classifier=classifier
            )

In [9]:
inp= data[200][0].unsqueeze(0)

In [10]:
loss = Lossfunction()

In [11]:
s = fd(inp)
kk=data[200][1]

In [12]:
for key in range(0,4):
    print(kk[key]['cls_targets'].shape)
    print(kk[key]['cls_targets'].sum())

torch.Size([4800, 1])
tensor(0)
torch.Size([19200, 1])
tensor(12)
torch.Size([76800, 1])
tensor(69)
torch.Size([307200, 1])
tensor(178)


In [55]:
transform = nn.Sequential(
    transforms.Resize((640, 640)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    )
ak = transform(train_dataset[100]['image']/255)
ak[0].mean() #0.0549
ak[1].mean() #0.0022
ak[2].mean() #-0.5621


tensor(-0.5621)

In [79]:
ak = transform(train_dataset[89]['image']/255).unsqueeze(0)
oak =model(ak)
ooak= torch.softmax(oak,dim=1)

In [80]:
oak

tensor([[-1.6657e+00, -1.0125e+00, -1.1825e+00, -2.6803e+00, -1.6418e+00,
         -2.7974e-01, -9.7216e-01,  1.2186e+00,  8.8198e-01, -1.2936e+00,
         -1.6477e+00, -1.4410e+00, -3.5058e-01, -1.5865e+00, -1.4154e+00,
         -1.0959e+00, -1.0114e+00, -2.5430e-01, -1.1257e+00, -8.2800e-01,
         -1.5392e+00, -6.1431e-01, -1.8899e+00,  4.7935e-01, -1.2840e+00,
         -1.6750e+00, -1.6561e+00, -1.9692e+00, -7.7283e-01, -9.5115e-01,
         -5.7824e-01, -1.2913e+00, -6.7403e-01, -6.7691e-01,  4.0566e-01,
         -3.1791e-01,  6.1518e-01, -6.2576e-01, -9.2850e-01,  2.6857e-01,
         -4.4177e-01, -5.1533e-01, -1.1532e+00, -4.2500e-03, -9.7136e-01,
         -1.1019e+00, -1.8175e-01, -2.2373e-01, -2.1664e+00, -1.6844e+00,
         -1.2037e+00,  7.7944e-01, -9.3508e-01, -1.0522e+00, -7.4613e-01,
         -1.5496e+00, -6.7422e-01, -1.1558e+00, -9.1632e-01, -2.8629e-01,
          2.2499e-01,  8.6036e-02,  1.9929e-01, -7.2600e-01, -6.4295e-01,
         -1.0848e+00, -6.0790e-01, -8.

In [82]:
(ooak)

tensor([[1.1139e-04, 2.1405e-04, 1.8058e-04, 4.0382e-05, 1.1408e-04, 4.4539e-04,
         2.2286e-04, 1.9927e-03, 1.4232e-03, 1.6160e-04, 1.1340e-04, 1.3944e-04,
         4.1494e-04, 1.2056e-04, 1.4306e-04, 1.9693e-04, 2.1429e-04, 4.5687e-04,
         1.9114e-04, 2.5742e-04, 1.2640e-04, 3.1875e-04, 8.9013e-05, 9.5151e-04,
         1.6316e-04, 1.1036e-04, 1.1246e-04, 8.2232e-05, 2.7202e-04, 2.2759e-04,
         3.3045e-04, 1.6197e-04, 3.0026e-04, 2.9940e-04, 8.8391e-04, 4.2871e-04,
         1.0899e-03, 3.1512e-04, 2.3281e-04, 7.7068e-04, 3.7877e-04, 3.5191e-04,
         1.8595e-04, 5.8666e-04, 2.2304e-04, 1.9574e-04, 4.9125e-04, 4.7105e-04,
         6.7514e-05, 1.0933e-04, 1.7679e-04, 1.2845e-03, 2.3128e-04, 2.0571e-04,
         2.7938e-04, 1.2510e-04, 3.0021e-04, 1.8548e-04, 2.3566e-04, 4.4249e-04,
         7.3781e-04, 6.4209e-04, 7.1909e-04, 2.8506e-04, 3.0975e-04, 1.9913e-04,
         3.2079e-04, 2.4605e-04, 1.4477e-04, 1.6455e-04, 7.3879e-05, 5.5637e-04,
         1.1748e-04, 5.5215e

In [50]:
ak.shape

torch.Size([3, 640, 640])

In [49]:
ak[2].mean()

tensor(-0.5621)

In [52]:
data[100][0].shape#.mean()

torch.Size([3, 640, 640])

In [14]:
s['3']['cls']

tensor([[[-1.2595],
         [-1.2492],
         [-1.2483],
         ...,
         [-1.2460],
         [-1.2495],
         [-1.2554]]], grad_fn=<ViewBackward0>)

In [15]:
(torch.sigmoid(s['3']['cls'])>0.5).sum()

tensor(0)

In [64]:
loss(s, kk)

box loss:0.4609232246875763
cls loss:0.38423848152160645


tensor(0.8452, device='cuda:0', grad_fn=<AddBackward0>)

In [67]:
loss(s, data[0][1])


box loss:0.7665533423423767
cls loss:7.646162986755371


tensor(8.4127, device='cuda:0', grad_fn=<AddBackward0>)

In [45]:
for i in list(fd.raw_features.keys()):
    print (fd.raw_features[i].shape)
    print (fd.raw_features[i].mean())
    

torch.Size([1, 24, 160, 160])
tensor(0.0034, grad_fn=<MeanBackward0>)
torch.Size([1, 32, 80, 80])
tensor(-0.0042, grad_fn=<MeanBackward0>)
torch.Size([1, 96, 40, 40])
tensor(0.0060, grad_fn=<MeanBackward0>)
torch.Size([1, 1280, 20, 20])
tensor(0.2237, grad_fn=<MeanBackward0>)


In [47]:
for i in list(fd.newfeatures.keys()):
    print (fd.newfeatures[i].shape)
    print (fd.newfeatures[i].mean())

torch.Size([1, 256, 20, 20])
tensor(-0.1097, grad_fn=<MeanBackward0>)
torch.Size([1, 256, 40, 40])
tensor(-0.1092, grad_fn=<MeanBackward0>)
torch.Size([1, 256, 80, 80])
tensor(-0.1187, grad_fn=<MeanBackward0>)
torch.Size([1, 256, 160, 160])
tensor(-0.1175, grad_fn=<MeanBackward0>)


In [42]:
loss(kk, data[0][1])


box loss:0.13568367063999176
cls loss:62838.42578125


tensor(62838.5625, grad_fn=<AddBackward0>)

In [52]:
model(inp).mean()

tensor(0.5135, grad_fn=<MeanBackward0>)